In [2]:
#Python script to train, evaluate and save model to estimate the Hurst exponent from trajectory of fBm.
# import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from stochastic.processes.continuous import FractionalBrownianMotion

### Data for training and testing

In [16]:
#generate our data for training and testing
nsamples = 100
ntimes = 14 #windowsize

def gen_fbm_data(nsamples,ntimes):
    """
    Function to produce fractional brownian motion data for neural network 
    training and testing
    Inputs: nsamples; number of samples, ntimes; number of times
    Outputs: traindata; training data for NN, trainlabels; labels associated 
            with traindata
    """
    data = np.empty((nsamples,ntimes))
    labels = np.empty((nsamples,1))
    for i in range(0,nsamples):
        hurst_exp = np.random.uniform(0.,1.)
        fbm = FractionalBrownianMotion(hurst=hurst_exp,t=1,rng=None)
        x = fbm.sample(ntimes)
        #apply differencing and normalization on the data
        dx = (x[1:]-x[0:-1])/(np.amax(x)-np.amin(x))
        data[i,:] = dx
        labels[i,:] = hurst_exp
        
    return data,labels

training_data,training_labels = gen_fbm_data(nsamples,ntimes)
test_data,test_labels = gen_fbm_data(nsamples,ntimes)
np.savetxt("H_testvalues_n"+str(ntimes)+".csv",test_labels,delimiter=",")

print('training data shape:',training_data.shape,'training labels shape:', training_labels.shape,
      'test data shape:',test_data.shape,'test labels shape:',test_labels.shape)

training data shape: (100, 14) training labels shape: (100, 1) test data shape: (100, 14) test labels shape: (100, 1)


### Developing neural network model

In [18]:
def gen_nn_model(ntimes,activation_func='relu',optimizer='adam',
                 loss_func='mean_absolute_error',summary=1):
    
    #create the model for a fully-connected network
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(ntimes,activation=activation_func,input_shape=(ntimes,)),
        tf.keras.layers.Dense(ntimes-1,activation=activation_func),
        tf.keras.layers.Dense(ntimes-2,activation=activation_func),
        tf.keras.layers.Dense(1,activation=activation_func)])
    #add optimizer, a loss function and metrics#
    # optimizer = tf.keras.optimizers.RMSprop(0.001)
    model.compile(optimizer=optimizer,
                  loss=loss_func,
                  metrics=[loss_func,'mean_squared_error'])
    if summary==1:
        model.summary()
    return model

model = gen_nn_model(ntimes)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 14)                210       
                                                                 
 dense_13 (Dense)            (None, 13)                195       
                                                                 
 dense_14 (Dense)            (None, 12)                168       
                                                                 
 dense_15 (Dense)            (None, 1)                 13        
                                                                 
Total params: 586
Trainable params: 586
Non-trainable params: 0
_________________________________________________________________


### Model training

In [11]:
def train_nn_model(nn_model,training_data,training_labels,n_epochs,validation_split=0.8,verbose=1):
    
    history = nn_model.fit(training_data,training_labels,epochs=n_epochs,
                        validation_split=validation_split,verbose=verbose)
    
    print("Saving model")
    nn_model.save("./model3dense_n"+str(ntimes)+".h5")
    del nn_model

train_nn_model(model,training_data,training_labels,100)
model = tf.keras.models.load_model("./model3dense_n"+str(ntimes)+".h5")

Epoch 1/100
1/1 [==============================] - 0s 440ms/step - loss: 0.0184 - mean_absolute_error: 0.0184 - mean_squared_error: 0.0022 - val_loss: 0.2185 - val_mean_absolute_error: 0.2185 - val_mean_squared_error: 0.0716
Epoch 2/100
1/1 [==============================] - 0s 48ms/step - loss: 0.0187 - mean_absolute_error: 0.0187 - mean_squared_error: 0.0022 - val_loss: 0.2185 - val_mean_absolute_error: 0.2185 - val_mean_squared_error: 0.0715
Epoch 3/100
1/1 [==============================] - 0s 51ms/step - loss: 0.0180 - mean_absolute_error: 0.0180 - mean_squared_error: 0.0021 - val_loss: 0.2187 - val_mean_absolute_error: 0.2187 - val_mean_squared_error: 0.0716
Epoch 4/100
1/1 [==============================] - 0s 46ms/step - loss: 0.0177 - mean_absolute_error: 0.0177 - mean_squared_error: 0.0021 - val_loss: 0.2188 - val_mean_absolute_error: 0.2188 - val_mean_squared_error: 0.0717
Epoch 5/100
1/1 [==============================] - 0s 43ms/step - loss: 0.0182 - mean_absolute_error: 0

1/1 [==============================] - 0s 34ms/step - loss: 0.0146 - mean_absolute_error: 0.0146 - mean_squared_error: 0.0014 - val_loss: 0.2169 - val_mean_absolute_error: 0.2169 - val_mean_squared_error: 0.0695
Epoch 38/100
1/1 [==============================] - 0s 38ms/step - loss: 0.0137 - mean_absolute_error: 0.0137 - mean_squared_error: 0.0014 - val_loss: 0.2170 - val_mean_absolute_error: 0.2170 - val_mean_squared_error: 0.0696
Epoch 39/100
1/1 [==============================] - 0s 35ms/step - loss: 0.0140 - mean_absolute_error: 0.0140 - mean_squared_error: 0.0013 - val_loss: 0.2166 - val_mean_absolute_error: 0.2166 - val_mean_squared_error: 0.0694
Epoch 40/100
1/1 [==============================] - 0s 55ms/step - loss: 0.0136 - mean_absolute_error: 0.0136 - mean_squared_error: 0.0013 - val_loss: 0.2166 - val_mean_absolute_error: 0.2166 - val_mean_squared_error: 0.0693
Epoch 41/100
1/1 [==============================] - 0s 40ms/step - loss: 0.0131 - mean_absolute_error: 0.0131 - m

1/1 [==============================] - 0s 45ms/step - loss: 0.0101 - mean_absolute_error: 0.0101 - mean_squared_error: 6.8681e-04 - val_loss: 0.2145 - val_mean_absolute_error: 0.2145 - val_mean_squared_error: 0.0673
Epoch 74/100
1/1 [==============================] - 0s 43ms/step - loss: 0.0105 - mean_absolute_error: 0.0105 - mean_squared_error: 7.0991e-04 - val_loss: 0.2145 - val_mean_absolute_error: 0.2145 - val_mean_squared_error: 0.0673
Epoch 75/100
1/1 [==============================] - 0s 42ms/step - loss: 0.0098 - mean_absolute_error: 0.0098 - mean_squared_error: 6.8369e-04 - val_loss: 0.2146 - val_mean_absolute_error: 0.2146 - val_mean_squared_error: 0.0675
Epoch 76/100
1/1 [==============================] - 0s 36ms/step - loss: 0.0099 - mean_absolute_error: 0.0099 - mean_squared_error: 6.4347e-04 - val_loss: 0.2145 - val_mean_absolute_error: 0.2145 - val_mean_squared_error: 0.0674
Epoch 77/100
1/1 [==============================] - 0s 38ms/step - loss: 0.0101 - mean_absolute_e

### Evaluating Model

In [24]:
def evaluate_nn_model(nn_model,test_data,test_labels,verbose=1):

    #evaluate the model generalizes by using the test data set
    loss, mae, mse = model.evaluate(test_data, test_labels, verbose=verbose)
    
    return loss,mae,mse

loss,mae,mse = evaluate_nn_model(model,test_data,test_labels)

print("Testing set Mean Abs Error: {:5.2f}".format(mae))

4/4 [==============================] - 0s 2ms/step - loss: 0.4516 - mean_absolute_error: 0.4516 - mean_squared_error: 0.2892
Testing set Mean Abs Error:  0.45


In [23]:
#predict values using data in the testing set
#test_predictions = model.predict(testdata)
#save predicted values
#np.savetxt("H_NNestimated_n"+str(ntimes)+".csv",test_predictions,delimiter=",")

def predict_h_labels(nn_model,test_data):
    
    #predict values using data in the testing set
    test_predictions = model.predict(test_data)
    #save predicted values
    np.savetxt("H_NNestimated_n"+str(ntimes)+".csv",test_predictions,delimiter=",")
    
    
predict_h_labels(model,test_data)

4/4 [==============================] - 0s 2ms/step


### Scrap

In [17]:

"""
#training data
traindata = np.empty((nsamples,ntimes))
trainlabels = np.empty((nsamples,1))
for i in range(0,nsamples):
    hurst_exp = np.random.uniform(0.,1.)
    fbm = FractionalBrownianMotion(hurst=hurst_exp,t=1,rng=None)
    x = fbm.sample(ntimes)
    #apply differencing and normalization on the data
    dx = (x[1:]-x[0:-1])/(np.amax(x)-np.amin(x))
    traindata[i,:] = dx
    trainlabels[i,:] = hurst_exp

#test data
testdata = np.empty((nsamples,ntimes))
testlabels = np.empty((nsamples,1))
for i in range(0,nsamples):
    hurst_exp = np.random.uniform(0.,1.)
    fbm = FractionalBrownianMotion(hurst=hurst_exp,t=1,rng=None)
    x = fbm.sample(ntimes)
    dx = (x[1:]-x[0:-1])/(np.amax(x)-np.amin(x))
    testdata[i,:] = dx
    testlabels[i,:] = hurst_exp
"""

"""
#create the model for a fully-connected network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(ntimes,activation='relu',input_shape=(ntimes,)),
    tf.keras.layers.Dense(ntimes-1,activation='relu'),
    tf.keras.layers.Dense(ntimes-2,activation='relu'),
    tf.keras.layers.Dense(1,activation='relu')
])
#add optimizer, a loss function and metrics#
optimizer = 'adam'
# optimizer = tf.keras.optimizers.RMSprop(0.001)
model.compile(optimizer=optimizer,
              loss='mean_absolute_error',
              metrics=['mean_absolute_error','mean_squared_error']
)
model.summary()
"""

"\n#create the model for a fully-connected network\nmodel = tf.keras.Sequential([\n    tf.keras.layers.Dense(ntimes,activation='relu',input_shape=(ntimes,)),\n    tf.keras.layers.Dense(ntimes-1,activation='relu'),\n    tf.keras.layers.Dense(ntimes-2,activation='relu'),\n    tf.keras.layers.Dense(1,activation='relu')\n])\n#add optimizer, a loss function and metrics#\noptimizer = 'adam'\n# optimizer = tf.keras.optimizers.RMSprop(0.001)\nmodel.compile(optimizer=optimizer,\n              loss='mean_absolute_error',\n              metrics=['mean_absolute_error','mean_squared_error']\n)\nmodel.summary()\n"